# Example notebook

In [1]:
%matplotlib inline
from skimage import transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
discrete_cmap()
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid

/home/kat/Projects/CubiCasa5k/floortrans/plotting.py:611: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)


In [2]:
rot = RotateNTurns()
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
print(len(room_classes))
print(len(icon_classes))

data_folder = 'data/cubicasa5k/'

12
11


In [3]:
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True, return_heatmaps=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
data_sample = next(iter(data_iter))
print(data_sample["folder"])
print(data_sample["label"][0].shape, 
      data_sample["label"][0][0].max().item(),
      data_sample["label"][0][1].max().item())

['/high_quality_architectural/1191/']
torch.Size([2, 592, 883]) 11 8


In [4]:
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]
model.conv4_ = torch.nn.Conv2d(256, n_classes, bias=True, kernel_size=1)
model.upsample = torch.nn.ConvTranspose2d(n_classes, n_classes, kernel_size=4, stride=4)
checkpoint = torch.load('model_best_val_loss_var.pkl')

model.load_state_dict(checkpoint['model_state'])
model.eval()
print("Model loaded.")

Model loaded.


In [5]:
normal_set = FloorplanSVG(data_folder, 'bogus_example.txt', format="txt", original_size=True, return_heatmaps=True)
data_loader = DataLoader(normal_set, batch_size=1)
next(iter(data_loader))

{'image': tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]]),
 'label': tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  .

In [6]:
def get_boggus_example(fmt):
    assert fmt in ("txt", "lmdb")
    normal_set = FloorplanSVG(data_folder, 'bogus_example.txt', format=fmt, original_size=True, return_heatmaps=True)
    data_loader = DataLoader(normal_set, batch_size=1)
    
    for d in data_loader:
        print(d["folder"])
        yield d

txt_sample_it = get_boggus_example("txt")
lmdb_sample_it = get_boggus_example("lmdb")
    
for txt_sample, lmdb_sample in zip(txt_sample_it, lmdb_sample_it):
    print(txt_sample["image"].shape)
    print(lmdb_sample["image"].shape)

    txt_label = txt_sample["label"][0].data.numpy()
    lmdb_label = lmdb_sample["label"][0].data.numpy()

    icons_idx = 1
    txt_label_icons = txt_label[icons_idx]
    lmdb_label_icons = lmdb_label[icons_idx]

    mask = np.where(lmdb_label_icons != txt_label_icons)
    print(txt_label_icons[mask])
    print(lmdb_label_icons[mask])
    
    print("\n\n")


['/high_quality_architectural/9365/']
[b'/high_quality_architectural/9365/']
torch.Size([1, 3, 941, 1131])
torch.Size([1, 3, 941, 1131])
[]
[]



['/high_quality/6555/']
[b'/high_quality/6555/']
torch.Size([1, 3, 690, 603])
torch.Size([1, 3, 690, 603])
[]
[]



['/high_quality/9775/']
[b'/high_quality/9775/']
torch.Size([1, 3, 681, 825])
torch.Size([1, 3, 681, 825])
[]
[]



['/high_quality/2057/']
[b'/high_quality/2057/']
torch.Size([1, 3, 285, 757])
torch.Size([1, 3, 285, 757])
[]
[]



['/high_quality_architectural/14503/']
[b'/high_quality_architectural/14503/']
torch.Size([1, 3, 728, 1658])
torch.Size([1, 3, 728, 1658])
[]
[]



['/high_quality_architectural/14138/']
[b'/high_quality_architectural/14138/']
torch.Size([1, 3, 782, 834])
torch.Size([1, 3, 782, 834])
[]
[]



['/high_quality_architectural/14029/']
[b'/high_quality_architectural/14029/']
torch.Size([1, 3, 918, 430])
torch.Size([1, 3, 918, 430])
[]
[]



['/high_quality_architectural/13251/']
[b'/high_quality_architectu

In [7]:
next(get_boggus_example("txt"))

['/high_quality/2057/']


{'image': tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]]),
 'label': tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  .

In [73]:
txt_sample["image"] = txt_sample["image"].squeeze()
txt_sample["label"] = txt_sample["label"].squeeze()

In [75]:
print(txt_sample["image"].shape)
print(txt_sample["label"].shape)

torch.Size([3, 1173, 691])
torch.Size([2, 1173, 691])


In [76]:
import lmdb 
import pickle 

with lmdb.open("data/cubicasa5k/cubi_lmdb/", map_size=int(200e9)) as env:
    filepath = '/high_quality_architectural/11525/'
    filepath_enc = filepath.encode("ascii")
    
    with env.begin(write=True) as trx:
        trx.replace(filepath_enc, pickle.dumps(txt_sample))

In [77]:
txt_sample = get_boggus_example("txt")
lmdb_sample = get_boggus_example("lmdb")
print(txt_sample["image"].shape)
print(lmdb_sample["image"].shape)

txt_label = txt_sample["label"][0].data.numpy()
lmdb_label = lmdb_sample["label"][0].data.numpy()

icons_idx = 1
txt_label_icons = txt_label[icons_idx]
lmdb_label_icons = lmdb_label[icons_idx]

mask = np.where(lmdb_label_icons != txt_label_icons)
print(txt_label_icons[mask])
print(lmdb_label_icons[mask])

['/high_quality_architectural/11525/']
[b'/high_quality_architectural/11525/']
torch.Size([1, 3, 1173, 691])
torch.Size([1, 3, 1173, 691])
[]
[]


In [ ]:
val = next(data_iter)
junctions = val['heatmaps']
folder = val['folder'][0]
image = val['image']
label = val['label']
print(label.size())
np_img = np.moveaxis(image[0].cpu().data.numpy(), 0, -1) / 2 + 0.5
plt.figure(figsize=(10,10))
plt.title('Source Image', fontsize=20)
plt.axis('off')
plt.imshow(np_img)
plt.show()

## The SVG floorplan
If the floorplan includes two floors the other floor might not be visible, but it will be parsed never the less.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= data_folder+folder[1:]+"model.svg", width=800)

## Parsed labels


### Draw the Rooms and Walls (according to the first labels)

In [ ]:
label_np = label.data.numpy()
# There are two "channels": 
# -- the label_np[0, 0] one contains the labels for the rooms
# -- the label_np[0, 1] second contains the labels for the icons
print(label_np.shape)
label_np = label_np[0]
print(label_np.shape)
print(label_np[0].max())
print(label_np[1].max())
plt.figure(figsize=(10,10))
ax = plt.subplot(1, 1, 1)
plt.title('Rooms and walls', fontsize=20)
ax.axis('off')

n_rooms = 12
rseg = ax.imshow(label_np[0], cmap='rooms', vmin=0, vmax=n_rooms-0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=20)
plt.show()

### Draw the icons 

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(1, 1, 1)
plt.title('Icons', fontsize=20)
ax.axis('off')
n_icons = 11
iseg = ax.imshow(label_np[1], cmap='icons', vmin=0, vmax=n_icons-0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=20)
plt.show()

In [ ]:
import seaborn as sns

(label_np[1] > 11).sum()

### Draw junctions (need heatmaps)

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(1, 1, 1)
plt.title('Wall junctions, Icon corners and opening end points', fontsize=20)
ax.axis('off')
ax.imshow(np_img)
h, w, _ = np_img.shape
draw_junction_from_dict(junctions, w, h, size=0.3, fontsize=10)
plt.show()

## Networks prediction for the segmentation

In [ ]:
with torch.no_grad():
    height = label_np.shape[1]
    width = label_np.shape[2]
    img_size = (height, width)
    
    rotations = [(0, 0), (1, -1), (2, 2), (-1, 1)]
    pred_count = len(rotations)
    prediction = torch.zeros([pred_count, n_classes, height, width])
    for i, r in enumerate(rotations):
        forward, back = r
        # We rotate first the image
        rot_image = rot(image, 'tensor', forward)
        pred = model(rot_image)
        # We rotate prediction back
        pred = rot(pred, 'tensor', back)
        # We fix heatmaps
        pred = rot(pred, 'points', back)
        # We make sure the size is correct
        pred = F.interpolate(pred, size=(height, width), mode='bilinear', align_corners=True)
        # We add the prediction to output
        prediction[i] = pred[0]

prediction = torch.mean(prediction, 0, True)
rooms_label = label_np[0]
icons_label = label_np[1]

rooms_pred = F.softmax(prediction[0, 21:21+12], 0).cpu().data.numpy()
rooms_pred = np.argmax(rooms_pred, axis=0)

icons_pred = F.softmax(prediction[0, 21+12:], 0).cpu().data.numpy()
icons_pred = np.argmax(icons_pred, axis=0)

plt.figure(figsize=(12,12))
ax = plt.subplot(1, 1, 1)
ax.axis('off')
rseg = ax.imshow(rooms_pred, cmap='rooms', vmin=0, vmax=n_rooms-0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=20)
plt.show()

plt.figure(figsize=(12,12))
ax = plt.subplot(1, 1, 1)
ax.axis('off')
iseg = ax.imshow(icons_pred, cmap='icons', vmin=0, vmax=n_icons-0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=20)
plt.show()

## Post-processed polygons

In [ ]:

heatmaps, rooms, icons = split_prediction(prediction, img_size, split)
polygons, types, room_polygons, room_types = get_polygons((heatmaps, rooms, icons), 0.2, [1, 2])

pol_room_seg, pol_icon_seg = polygons_to_image(polygons, types, room_polygons, room_types, height, width)
plt.figure(figsize=(12,12))
ax = plt.subplot(1, 1, 1)
ax.axis('off')
rseg = ax.imshow(pol_room_seg, cmap='rooms', vmin=0, vmax=n_rooms-0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=20)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,12))
ax = plt.subplot(1, 1, 1)
ax.axis('off')
iseg = ax.imshow(pol_icon_seg, cmap='icons', vmin=0, vmax=n_icons-0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=20)
plt.tight_layout()
plt.show()

## Ground truth and prediction comparisons

In [ ]:
fig = plt.figure(figsize=(26, 12))
grid = AxesGrid(fig, 111,
                nrows_ncols=(1, 2),
                axes_pad=0.05,
                cbar_mode='single',
                cbar_location='right',
                cbar_pad=0.1
                )
images = [label_np[0], pol_room_seg]
for i, ax in enumerate(grid):
    ax.set_axis_off()
    im = ax.imshow(images[i], cmap='rooms', vmin=0, vmax=n_rooms-0.1)
cbar = ax.cax.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5)
cbar.ax.set_yticklabels(room_classes, fontsize=26)
plt.show()

In [ ]:
fig = plt.figure(figsize=(26, 12))
grid = AxesGrid(fig, 111,
                nrows_ncols=(1, 2),
                axes_pad=0.05,
                cbar_mode='single',
                cbar_location='right',
                cbar_pad=0.1
                )

images = [label_np[1], pol_icon_seg]
for i, ax in enumerate(grid):
    ax.set_axis_off()
    im = ax.imshow(images[i], cmap='icons', vmin=0, vmax=n_icons-0.1)

cbar = ax.cax.colorbar(iseg, ticks=np.arange(n_icons) + 0.5)
cbar.ax.set_yticklabels(icon_classes, fontsize=26)
plt.show()